In [1]:
import os
import shutil
import tempfile
%pip install hydromt_sfincs
import hydromt_sfincs
from pathlib import Path

import numpy as np
import xarray as xr
import rioxarray
import gcsfs
import pandas as pd

from hydromt_sfincs import SfincsModel

print("✅ Imports complete")

  Using cached hydromt_sfincs-1.2.2-py3-none-any.whl.metadata (7.4 kB)
  Using cached hydromt-0.10.1-py3-none-any.whl.metadata (9.3 kB)
  Using cached pyflwdir-0.5.10-py3-none-any.whl.metadata (4.3 kB)
  Using cached xugrid-0.14.3-py3-none-any.whl.metadata (3.6 kB)
  Using cached tomli_w-1.2.0-py3-none-any.whl.metadata (5.7 kB)
  Using cached universal_pathlib-0.3.8-py3-none-any.whl.metadata (39 kB)
  Using cached xmltodict-1.0.2-py3-none-any.whl.metadata (15 kB)
  Using cached numba_celltree-0.4.1-py3-none-any.whl.metadata (8.0 kB)
  Using cached pathlib_abc-0.5.2-py3-none-any.whl.metadata (4.6 kB)
Using cached hydromt_sfincs-1.2.2-py3-none-any.whl (121 kB)
Using cached hydromt-0.10.1-py3-none-any.whl (222 kB)
Using cached xugrid-0.14.3-py3-none-any.whl (145 kB)
Using cached numba_celltree-0.4.1-py3-none-any.whl (42 kB)
Using cached pyflwdir-0.5.10-py3-none-any.whl (66 kB)
Using cached universal_pathlib-0.3.8-py3-none-any.whl (82 kB)
Using cached pathlib_abc-0.5.2-py3-none-any.whl (19

In [2]:
# ============================================================
# Configuration - Define paths to input data
# UPDATED: Rain forcing now uses NetCDF FEWS format
# ============================================================

SCRATCH_BUCKET = "gs://leap-scratch/renriviera"
OUT_PREFIX = f"{SCRATCH_BUCKET.rstrip('/')}/sfincs_soundview_preproc"

# Target parameters
TARGET_CRS = "EPSG:26918"  # UTM Zone 18N for NYC
TARGET_RES = 25  # meters
YEAR = 2025

# Raster inputs (from elevation notebook)
PATHS_GCS = {
    "dep_dem":      f"{OUT_PREFIX}/rasters/final/dep_dem_utm{TARGET_RES}m_meters.tif",
    "landcover":    f"{OUT_PREFIX}/rasters/clean/landcover_worldcover_utm{TARGET_RES}m.tif",
    "manning":      f"{OUT_PREFIX}/rasters/final/manning_n_utm{TARGET_RES}m.tif",
    "impervious":   f"{OUT_PREFIX}/rasters/final/impervious_frac_utm{TARGET_RES}m.tif",
    "curve_number": f"{OUT_PREFIX}/rasters/final/curve_number_cn_utm{TARGET_RES}m.tif",
}

# Wind forcing - FEWS NetCDF (from windspeed notebook)
WIND_GCS = f"{OUT_PREFIX}/forcing/wind/sfincs_netamuamv_hrrr_u10v10_soundview_{YEAR}.nc"

# UPDATED: Rain forcing - FEWS NetCDF (from corrected rain notebook)
# Changed from ASCII to NetCDF format to match windspeed conventions
RAIN_GCS = f"{OUT_PREFIX}/forcing/rain_hrrr/sfincs_rain_hrrr_soundview_{YEAR}.nc"

print("Configuration:")
print(f"  OUT_PREFIX: {OUT_PREFIX}")
print(f"  CRS: {TARGET_CRS}")
print(f"  Resolution: {TARGET_RES}m")
print(f"  Year: {YEAR}")
print(f"\nInput files:")
print(f"  DEM: {PATHS_GCS['dep_dem']}")
print(f"  Wind: {WIND_GCS}")
print(f"  Rain: {RAIN_GCS}")

Configuration:
  OUT_PREFIX: gs://leap-scratch/renriviera/sfincs_soundview_preproc
  CRS: EPSG:26918
  Resolution: 25m
  Year: 2025

Input files:
  DEM: gs://leap-scratch/renriviera/sfincs_soundview_preproc/rasters/final/dep_dem_utm25m_meters.tif
  Wind: gs://leap-scratch/renriviera/sfincs_soundview_preproc/forcing/wind/sfincs_netamuamv_hrrr_u10v10_soundview_2025.nc
  Rain: gs://leap-scratch/renriviera/sfincs_soundview_preproc/forcing/rain_hrrr/sfincs_rain_hrrr_soundview_2025.nc


In [3]:
# ============================================================
# Download model inputs locally (AUTHENTICATED gcsfs)
# ============================================================

import tempfile
from pathlib import Path
import gcsfs

cache_dir = Path(tempfile.mkdtemp(prefix="sfincs_run_inputs_"))
print("Local input cache:", cache_dir)

def make_gcsfs():
    """Initialize GCS filesystem with authentication"""
    for tok in ["google_default", "cloud"]:
        try:
            fs = gcsfs.GCSFileSystem(token=tok)
            # Validate auth works
            _ = fs.ls(SCRATCH_BUCKET.replace("gs://", ""), detail=False)[:1]
            print(f"✅ Authenticated with token='{tok}'")
            return fs
        except Exception as e:
            print(f"⚠️ token='{tok}' failed: {type(e).__name__}")
    
    # Fallback
    print("⚠️ Falling back to anonymous (may fail for private buckets)")
    return gcsfs.GCSFileSystem(token="anon")

fs_gcs = make_gcsfs()

def strip_gs(path: str) -> str:
    return path.replace("gs://", "") if path.startswith("gs://") else path

def gcs_exists(gs_path: str) -> bool:
    return fs_gcs.exists(strip_gs(gs_path))

def gcs_to_local(gs_path: str, local_path: Path, recursive: bool = False):
    gcs_rel = strip_gs(gs_path)
    local_path.parent.mkdir(parents=True, exist_ok=True)
    
    if not fs_gcs.exists(gcs_rel):
        raise FileNotFoundError(f"GCS path not accessible: {gs_path}")
    
    fs_gcs.get(gcs_rel, str(local_path), recursive=recursive)
    
    if not local_path.exists():
        raise FileNotFoundError(f"Download failed: {gs_path}")
    
    return local_path

# Download rasters
paths_local = {}
print("\n--- Downloading rasters ---")
for k, p in PATHS_GCS.items():
    out = cache_dir / "rasters" / Path(p).name
    paths_local[k] = gcs_to_local(p, out, recursive=False)
    size_kb = paths_local[k].stat().st_size / 1024
    print(f"  ✅ {k:12s} ({size_kb:>8.1f} KB)")

# Download wind forcing (NetCDF)
print("\n--- Downloading wind forcing ---")
wind_local = cache_dir / "forcing" / "wind.nc"
wind_local = gcs_to_local(WIND_GCS, wind_local, recursive=False)
size_mb = wind_local.stat().st_size / (1024*1024)
print(f"  ✅ Wind ({size_mb:.1f} MB)")

# Download rain forcing (NetCDF)
print("\n--- Downloading rain forcing ---")
rain_local = cache_dir / "forcing" / "rain.nc"
rain_local = gcs_to_local(RAIN_GCS, rain_local, recursive=False)
size_mb = rain_local.stat().st_size / (1024*1024)
print(f"  ✅ Rain ({size_mb:.1f} MB)")

print(f"\n✅ All inputs cached in: {cache_dir}")

Local input cache: /tmp/sfincs_run_inputs_jpgvf_pk
✅ Authenticated with token='google_default'

--- Downloading rasters ---
  ✅ dep_dem      (    59.3 KB)
  ✅ landcover    (     4.6 KB)
  ✅ manning      (     5.4 KB)
  ✅ impervious   (     5.2 KB)
  ✅ curve_number (     5.1 KB)

--- Downloading wind forcing ---
  ✅ Wind (0.2 MB)

--- Downloading rain forcing ---
  ✅ Rain (0.1 MB)

✅ All inputs cached in: /tmp/sfincs_run_inputs_jpgvf_pk


In [4]:
# ============================================================
# Initialize SFINCS model object
# ============================================================

workdir = Path(tempfile.mkdtemp(prefix="sfincs_model_"))
print("Model directory:", workdir)

m = SfincsModel(root=str(workdir), mode="w")
print("✅ SFINCS model initialized")

# Debug info
print(f"  Model root: {m.root}")
print(f"  Has geoms: {hasattr(m, 'geoms')}")
print(f"  Has grid: {hasattr(m, 'grid')}")
print(f"  Has config: {hasattr(m, 'config')}")

Model directory: /tmp/sfincs_model_v5mi3bfu
✅ SFINCS model initialized
  Model root: /tmp/sfincs_model_v5mi3bfu
  Has geoms: True
  Has grid: True
  Has config: True


In [5]:
# ============================================================
# Setup region + grid from DEM
# ============================================================

import geopandas as gpd
from shapely.geometry import box
import inspect

# Load DEM
dem_fn = str(paths_local["dep_dem"])
print("Loading DEM:", dem_fn)
dem = rioxarray.open_rasterio(dem_fn).squeeze()

if dem.rio.crs is None:
    raise RuntimeError("DEM has no CRS")

crs = dem.rio.crs
res_x, res_y = dem.rio.resolution()
res = float(abs(res_x))
minx, miny, maxx, maxy = map(float, dem.rio.bounds())

print(f"\n✅ DEM loaded")
print(f"  Shape: {dem.shape}")
print(f"  CRS: {crs}")
print(f"  Resolution: {res}m")
print(f"  Bounds: [{minx:.1f}, {miny:.1f}, {maxx:.1f}, {maxy:.1f}]")

# Create region geometry
region_poly = box(minx, miny, maxx, maxy)
region_gdf = gpd.GeoDataFrame(
    {"name": ["soundview"]},
    geometry=[region_poly],
    crs=crs
)

# Setup region
m.setup_region(region={"geom": region_gdf})
print("\n✅ Region setup complete")

# Setup grid from region
try:
    m.setup_grid_from_region(
        region={"geom": region_gdf},
        res=res,
        crs=str(crs),
        align=True,
    )
    print("✅ Grid setup complete")
except Exception as e:
    print(f"⚠️ setup_grid_from_region failed: {type(e).__name__}")

# Check grid
grid = m.grid
print(f"\nGrid status:")
print(f"  Dimensions: {dict(grid.sizes)}")

# Force grid if empty
if len(grid.dims) == 0:
    print("\n⚠️ Grid empty, forcing from DEM coordinates")
    
    grid_fallback = xr.Dataset(
        coords={
            "x": dem["x"].astype("float64"),
            "y": dem["y"].astype("float64"),
        }
    )
    grid_fallback = grid_fallback.rio.write_crs(crs)
    grid_fallback = grid_fallback.rio.set_spatial_dims(x_dim="x", y_dim="y")
    
    setattr(m, "_grid", grid_fallback)
    grid = m.grid
    print(f"✅ Grid forced: {dict(grid.sizes)}")

# Validate grid
try:
    grid.raster.set_spatial_dims(x_dim="x", y_dim="y")
    print(f"\n✅ Grid validated")
    print(f"  CRS: {grid.raster.crs}")
    print(f"  Bounds: {grid.raster.bounds}")
except Exception as e:
    print(f"\n⚠️ Grid raster accessor not available: {type(e).__name__}")

Loading DEM: /tmp/sfincs_run_inputs_jpgvf_pk/rasters/dep_dem_utm25m_meters.tif

✅ DEM loaded
  Shape: (131, 137)
  CRS: EPSG:26918
  Resolution: 25.0m
  Bounds: [594250.0, 4517925.0, 597675.0, 4521200.0]

✅ Region setup complete
✅ Grid setup complete

Grid status:
  Dimensions: {}

⚠️ Grid empty, forcing from DEM coordinates
✅ Grid forced: {'x': 137, 'y': 131}

✅ Grid validated
  CRS: PROJCS["NAD83 / UTM zone 18N",GEOGCS["NAD83",DATUM["North_American_Datum_1983",SPHEROID["GRS 1980",6378137,298.257222101,AUTHORITY["EPSG","7019"]],AUTHORITY["EPSG","6269"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4269"]],PROJECTION["Transverse_Mercator"],PARAMETER["latitude_of_origin",0],PARAMETER["central_meridian",-75],PARAMETER["scale_factor",0.9996],PARAMETER["false_easting",500000],PARAMETER["false_northing",0],UNIT["metre",1,AUTHORITY["EPSG","9001"]],AXIS["Easting",EAST],AXIS["Northing",NORTH],AUTHORITY["EPSG","26918

In [6]:
# ============================================================
# Setup elevation and active mask
# ============================================================

# Setup elevation
dep_path = str(paths_local["dep_dem"])
datasets_dep = [{"elevtn": dep_path, "name": "dem_meters"}]

m.setup_dep(
    datasets_dep=datasets_dep,
    interp_method="linear",
)
print("✅ Elevation setup complete")

# Create active mask from bbox
bbox_poly = box(minx, miny, maxx, maxy)
bbox_gdf = gpd.GeoDataFrame(
    {"name": ["active_bbox"]},
    geometry=[bbox_poly],
    crs=crs
)

m.setup_mask_active(mask=bbox_gdf, reset_mask=True)
print("✅ Active mask setup complete")

if hasattr(m, "mask"):
    active_cells = (m.mask == 1).sum().values
    print(f"  Active cells: {active_cells:,}")

✅ Elevation setup complete
✅ Active mask setup complete
  Active cells: 17,947


In [7]:
# ============================================================
# Setup Manning roughness
# ============================================================

manning_fn = str(paths_local["manning"])
datasets_rgh = [{"manning": manning_fn}]

m.setup_manning_roughness(datasets_rgh=datasets_rgh)
print("✅ Manning roughness setup complete")

✅ Manning roughness setup complete


In [8]:
# ============================================================
# Setup infiltration (Curve Number)
# ============================================================

import inspect

cn_fn = str(paths_local["curve_number"])
print(f"Using CN raster: {cn_fn}")

# Validate CN file
cn_da = rioxarray.open_rasterio(cn_fn, masked=False).squeeze()
print(f"  CN shape: {cn_da.shape}")
print(f"  CN CRS: {cn_da.rio.crs}")

# Try to setup CN infiltration
sig = inspect.signature(m.setup_cn_infiltration)

try:
    if "cn" in sig.parameters:
        m.setup_cn_infiltration(cn=cn_fn)
    else:
        m.setup_cn_infiltration(cn_fn)
    print("✅ CN infiltration setup complete")
    
except Exception as e:
    print(f"⚠️ CN setup failed: {type(e).__name__}")
    print("Using constant infiltration fallback...")
    
    qinf_ms = 1.4e-6  # ~5 mm/hr
    sig2 = inspect.signature(m.setup_constant_infiltration)
    
    if "qinf" in sig2.parameters:
        m.setup_constant_infiltration(qinf=qinf_ms)
    else:
        m.setup_constant_infiltration(qinf_ms)
    
    print(f"✅ Constant infiltration set: {qinf_ms:.2e} m/s")

Using CN raster: /tmp/sfincs_run_inputs_jpgvf_pk/rasters/curve_number_cn_utm25m.tif
  CN shape: (131, 137)
  CN CRS: EPSG:26918
✅ CN infiltration setup complete


In [9]:
# ============================================================
# Load and validate RAIN forcing (FEWS NetCDF)
# CRITICAL: Updated from ASCII to NetCDF format
# ============================================================

print(f"Loading rain forcing: {rain_local}")

# Open rain NetCDF
ds_rain = xr.open_dataset(rain_local, engine="netcdf4")

print("\n--- Rain Dataset ---")
print(ds_rain)

# Verify required variable
if "precip" not in ds_rain:
    raise RuntimeError(
        f"Rain dataset must contain 'precip' variable. "
        f"Found: {list(ds_rain.data_vars)}"
    )

# Verify dimensions
if not {"time", "y", "x"}.issubset(ds_rain.dims):
    raise RuntimeError(
        f"Rain must have (time, y, x) dimensions. "
        f"Found: {dict(ds_rain.sizes)}"
    )

# Verify time is datetime64
if not np.issubdtype(ds_rain["time"].dtype, np.datetime64):
    raise RuntimeError(
        f"Rain time must be datetime64. "
        f"Got: {ds_rain['time'].dtype}"
    )

# CRITICAL: Verify time is in correct year (not 1970!)
first_time = pd.Timestamp(ds_rain.time.values[0])
last_time = pd.Timestamp(ds_rain.time.values[-1])
first_year = first_time.year

print(f"\n✅ Rain time validation:")
print(f"  First timestamp: {first_time}")
print(f"  Last timestamp: {last_time}")
print(f"  Year: {first_year}")

if first_year != YEAR:
    raise RuntimeError(
        f"❌ RAIN FORCING YEAR MISMATCH!\n"
        f"Expected: {YEAR}\n"
        f"Got: {first_year}\n"
        f"First time: {first_time}\n\n"
        f"This means SFINCS would interpret your rain data as being from {first_year}!\n"
        f"Fix: Regenerate rain forcing with correct FEWS time format."
    )

print(f"\n✅ Rain dataset validated")
print(f"  Time steps: {len(ds_rain.time)}")
print(f"  Duration: {(last_time - first_time).days} days")
print(f"  Grid: {ds_rain.sizes['y']} x {ds_rain.sizes['x']}")
print(f"  Max precip: {float(ds_rain['precip'].max()):.2f} mm/hr")

# Attach CRS metadata for HydroMT
ds_rain.attrs["crs"] = str(TARGET_CRS)
ds_rain.raster.set_spatial_dims(x_dim="x", y_dim="y")
ds_rain.raster.set_crs(TARGET_CRS)

print(f"✅ Rain CRS metadata attached")

Loading rain forcing: /tmp/sfincs_run_inputs_jpgvf_pk/forcing/rain.nc

--- Rain Dataset ---
<xarray.Dataset> Size: 193kB
Dimensions:  (time: 8040, y: 2, x: 2)
Coordinates:
  * time     (time) datetime64[ns] 64kB 2025-01-01 ... 2025-12-01T23:00:00
  * y        (y) float64 16B 4.515e+06 4.525e+06
  * x        (x) float64 16B 5.91e+05 6.01e+05
Data variables:
    precip   (time, y, x) float32 129kB ...
Attributes:
    crs:      EPSG:26918

✅ Rain time validation:
  First timestamp: 2025-01-01 00:00:00
  Last timestamp: 2025-12-01 23:00:00
  Year: 2025

✅ Rain dataset validated
  Time steps: 8040
  Duration: 334 days
  Grid: 2 x 2
  Max precip: 28.50 mm/hr
✅ Rain CRS metadata attached


In [10]:
# ============================================================
# Load and validate WIND forcing (FEWS NetCDF)
# ============================================================

print(f"Loading wind forcing: {wind_local}")

# Open wind NetCDF
ds_wind = xr.open_dataset(wind_local, engine="netcdf4")

print("\n--- Wind Dataset ---")
print(ds_wind)

# Rename variables if needed
rename_map = {}
if "amu" in ds_wind and "wind10_u" not in ds_wind:
    rename_map["amu"] = "wind10_u"
if "amv" in ds_wind and "wind10_v" not in ds_wind:
    rename_map["amv"] = "wind10_v"

if rename_map:
    ds_wind = ds_wind.rename(rename_map)
    print(f"✅ Renamed variables: {rename_map}")

# Verify required variables
if "wind10_u" not in ds_wind or "wind10_v" not in ds_wind:
    raise RuntimeError(
        f"Wind dataset must contain wind10_u and wind10_v. "
        f"Found: {list(ds_wind.data_vars)}"
    )

# Verify dimensions
if not {"time", "y", "x"}.issubset(ds_wind.dims):
    raise RuntimeError(
        f"Wind must have (time, y, x) dimensions. "
        f"Found: {dict(ds_wind.sizes)}"
    )

# Ensure float32
ds_wind["wind10_u"] = ds_wind["wind10_u"].astype("float32")
ds_wind["wind10_v"] = ds_wind["wind10_v"].astype("float32")

# Attach CRS metadata
ds_wind.attrs["crs"] = str(TARGET_CRS)
ds_wind.raster.set_spatial_dims(x_dim="x", y_dim="y")
ds_wind.raster.set_crs(TARGET_CRS)

print(f"\n✅ Wind dataset validated")
print(f"  Time steps: {len(ds_wind.time)}")
print(f"  Time range: {ds_wind.time.values[0]} to {ds_wind.time.values[-1]}")
print(f"  Grid: {ds_wind.sizes['y']} x {ds_wind.sizes['x']}")

# Check time overlap with rain
wind_t0 = pd.Timestamp(ds_wind.time.values[0])
wind_t1 = pd.Timestamp(ds_wind.time.values[-1])
rain_t0 = pd.Timestamp(ds_rain.time.values[0])
rain_t1 = pd.Timestamp(ds_rain.time.values[-1])

overlap_start = max(wind_t0, rain_t0)
overlap_end = min(wind_t1, rain_t1)

if overlap_end < overlap_start:
    raise RuntimeError(
        f"❌ NO TIME OVERLAP between wind and rain!\n"
        f"Wind: {wind_t0} to {wind_t1}\n"
        f"Rain: {rain_t0} to {rain_t1}\n"
    )

print(f"\n✅ Time overlap validated")
print(f"  Overlap: {overlap_start} to {overlap_end}")
print(f"  Duration: {(overlap_end - overlap_start).days} days")

Loading wind forcing: /tmp/sfincs_run_inputs_jpgvf_pk/forcing/wind.nc

--- Wind Dataset ---
<xarray.Dataset> Size: 312kB
Dimensions:  (time: 7800, y: 2, x: 2)
Coordinates:
  * time     (time) datetime64[ns] 62kB 2025-01-01 ... 2025-11-21T23:00:00
  * y        (y) float64 16B 4.52e+06 4.523e+06
  * x        (x) float64 16B 5.949e+05 5.979e+05
Data variables:
    amu      (time, y, x) float32 125kB ...
    amv      (time, y, x) float32 125kB ...
Attributes:
    crs:      EPSG:26918
✅ Renamed variables: {'amu': 'wind10_u', 'amv': 'wind10_v'}

✅ Wind dataset validated
  Time steps: 7800
  Time range: 2025-01-01T00:00:00.000000000 to 2025-11-21T23:00:00.000000000
  Grid: 2 x 2

✅ Time overlap validated
  Overlap: 2025-01-01 00:00:00 to 2025-11-21 23:00:00
  Duration: 324 days


In [11]:
# ============================================================
# Cell 11 (FIXED): Attach forcing data to model
# SOLUTION: Convert rain NetCDF to ASCII for spatially uniform precip
# ============================================================

import tempfile
from pathlib import Path

# Check spatial dimensions of forcing datasets
print("Checking spatial dimensions...")
print(f"  Wind grid: {ds_wind.sizes['y']} x {ds_wind.sizes['x']}")
print(f"  Rain grid: {ds_rain.sizes['y']} x {ds_rain.sizes['x']}")

# Set simulation timeframe from overlap period
if hasattr(m, "config") and isinstance(m.config, dict):
    m.config["tref"] = overlap_start.strftime("%Y%m%d %H%M%S")
    m.config["tstart"] = overlap_start.strftime("%Y%m%d %H%M%S")
    m.config["tstop"] = overlap_end.strftime("%Y%m%d %H%M%S")
    print("\n✅ Simulation timeframe set")
    print(f"  Start: {overlap_start}")
    print(f"  End: {overlap_end}")
    print(f"  Duration: {(overlap_end - overlap_start).days} days")

# ============================================================
# WIND FORCING: Use grid-based method if 2D, otherwise skip
# ============================================================

if ds_wind.sizes['y'] >= 2 and ds_wind.sizes['x'] >= 2:
    print("\n✅ Wind has proper 2D grid, attaching...")
    m.setup_wind_forcing_from_grid(wind=ds_wind)
    print("✅ Wind forcing attached")
else:
    print(f"\n⚠️ Wind grid too small ({ds_wind.sizes['y']}x{ds_wind.sizes['x']})")
    print("Skipping wind forcing (or convert to ASCII format)")

# ============================================================
# RAIN FORCING: Convert to ASCII if 1x1, otherwise use grid
# ============================================================

if ds_rain.sizes['y'] >= 2 and ds_rain.sizes['x'] >= 2:
    # Rain has proper 2D grid - use grid-based method
    print("\n✅ Rain has proper 2D grid, attaching...")
    m.setup_precip_forcing_from_grid(precip=ds_rain)
    print("✅ Rain forcing attached (grid-based)")
    
else:
    # Rain is 1x1 - convert to ASCII format (spatially uniform)
    print(f"\n⚠️ Rain grid is {ds_rain.sizes['y']}x{ds_rain.sizes['x']} (too small for grid method)")
    print("Skipping")

print("\n✅ All forcing attached")
if hasattr(m, "forcing"):
    print(f"  Forcing variables: {list(m.forcing.keys())}")

Checking spatial dimensions...
  Wind grid: 2 x 2
  Rain grid: 2 x 2

✅ Simulation timeframe set
  Start: 2025-01-01 00:00:00
  End: 2025-11-21 23:00:00
  Duration: 324 days

✅ Wind has proper 2D grid, attaching...
✅ Wind forcing attached

✅ Rain has proper 2D grid, attaching...
✅ Rain forcing attached (grid-based)

✅ All forcing attached
  Forcing variables: ['wind10_u', 'wind10_v', 'precip_2d']


In [12]:
# ============================================================
# Configure simulation parameters
# ============================================================

# Timestep and output settings
m.config["dt"] = 60  # 1 minute timestep
m.config["tspinup"] = 60  # 60 minute spinup
m.config["outputformat"] = "netcdf"
m.config["dtout"] = 3600  # Output every hour
m.config["dthisout"] = 0
m.config["dtmaxout"] = 0

# Physics settings
m.config["advection"] = 1
m.config["viscosity"] = 1
m.config["manning"] = 1  # Spatially varying
m.config["infiltration"] = 1  # CN-based
m.config["alpha"] = 0.75
m.config["theta"] = 1.0

print("✅ Simulation configuration complete")
print(f"  Timestep: {m.config['dt']} seconds")
print(f"  Output interval: {m.config['dtout']} seconds")
print(f"  Manning: {'variable' if m.config['manning'] == 1 else 'constant'}")
print(f"  Infiltration: {'enabled' if m.config['infiltration'] == 1 else 'disabled'}")

✅ Simulation configuration complete
  Timestep: 60 seconds
  Output interval: 3600 seconds
  Manning: variable
  Infiltration: enabled


In [13]:
# ============================================================
# Write model files
# ============================================================

print(f"Writing model files to: {m.root}")
m.write()

print("\n✅ Model files written successfully!")

# List generated files
print("\n📁 Model files:")
for f in sorted(Path(m.root).glob("*")):
    if f.is_file():
        size_kb = f.stat().st_size / 1024
        print(f"  {f.name:30s} {size_kb:>10.1f} KB")

print(f"\n📍 Model directory: {m.root}")
print("\n✅ Model is ready to run!")

Writing model files to: /tmp/sfincs_model_v5mi3bfu

✅ Model files written successfully!

📁 Model files:
  hydromt.log                           0.5 KB
  precip_2d.nc                        199.6 KB
  sfincs.dep                           70.1 KB
  sfincs.ind                           70.1 KB
  sfincs.inp                            1.4 KB
  sfincs.man                           70.1 KB
  sfincs.msk                           17.5 KB
  sfincs.scs                           70.1 KB
  wind_2d.nc                          322.5 KB

📍 Model directory: /tmp/sfincs_model_v5mi3bfu

✅ Model is ready to run!


In [14]:
# ============================================================
# Run SFINCS simulation (OPTIONAL)
# Requires SFINCS executable
# ============================================================

import subprocess

print("🚀 Starting SFINCS simulation...")
print(f"Working directory: {m.root}")
print("\n⏱️ This may take 30-90 minutes...\n")

%conda install -c conda-forge sfincs

try:
    result = subprocess.run(
        ["sfincs"],
        cwd=m.root,
        capture_output=True,
        text=True,
        timeout=7200  # 2 hour timeout
    )
    
    print("="*60)
    print("SFINCS OUTPUT:")
    print("="*60)
    print(result.stdout)
    
    if result.returncode != 0:
        print("\n" + "="*60)
        print("SFINCS ERRORS:")
        print("="*60)
        print(result.stderr)
        raise RuntimeError(f"SFINCS failed with exit code {result.returncode}")
    
    print("\n" + "="*60)
    print("✅ SIMULATION COMPLETE!")
    print("="*60)
    
    # Check output files
    output_files = list(Path(m.root).glob("sfincs_*.nc"))
    if output_files:
        print(f"\n📊 Output files ({len(output_files)}):")
        for f in output_files:
            size_mb = f.stat().st_size / (1024 * 1024)
            print(f"  {f.name}: {size_mb:.1f} MB")
    
except FileNotFoundError:
    print("="*60)
    print("❌ SFINCS executable not found")
    print("="*60)
    print("\nTo install: conda install -c conda-forge sfincs")
    print(f"Model files ready in: {m.root}")
    
except subprocess.TimeoutExpired:
    print("❌ Simulation timed out after 2 hours")
    
except Exception as e:
    print(f"❌ Error: {e}")
    raise

🚀 Starting SFINCS simulation...
Working directory: /tmp/sfincs_model_v5mi3bfu

⏱️ This may take 30-90 minutes...

Channels:
 - conda-forge
Platform: linux-64
Solving environment: failed

PackagesNotFoundError: The following packages are not available from current channels:

  - sfincs

Current channels:

  - https://conda.anaconda.org/conda-forge

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.



Note: you may need to restart the kernel to use updated packages.
❌ SFINCS executable not found

To install: conda install -c conda-forge sfincs
Model files ready in: /tmp/sfincs_model_v5mi3bfu


In [15]:
# ============================================================
# Visualize results (OPTIONAL)
# ============================================================

import matplotlib.pyplot as plt

output_file = Path(m.root) / "sfincs_map.nc"

if not output_file.exists():
    print("⚠️ No simulation output found")
    print("Run the previous cell to execute the simulation")
else:
    print(f"📊 Loading results: {output_file.name}")
    
    ds_results = xr.open_dataset(output_file)
    print("\n--- Results Dataset ---")
    print(ds_results)
    
    if "zs" in ds_results:
        # Water surface elevation
        zs = ds_results["zs"]
        max_wl = zs.max(dim="time")
        
        # Calculate flood depth
        max_depth = max_wl - dem
        max_depth = max_depth.where(max_depth > 0, 0)
        
        print(f"\n📈 Flood Statistics:")
        print(f"  Max water level: {float(max_wl.max()):.3f} m")
        print(f"  Max flood depth: {float(max_depth.max()):.3f} m")
        
        # Calculate flooded area
        flooded_cells = (max_depth > 0.05).sum().values
        flooded_area_km2 = flooded_cells * (TARGET_RES ** 2) / 1e6
        print(f"  Flooded area (>5cm): {flooded_area_km2:.3f} km²")
        
        # Plot
        fig, ax = plt.subplots(figsize=(12, 8))
        max_depth.plot(
            ax=ax,
            cmap="Blues",
            vmin=0,
            vmax=2.0,
            cbar_kwargs={"label": "Maximum Flood Depth (m)"}
        )
        ax.set_title(f"Maximum Flood Depth - Soundview {YEAR}")
        plt.tight_layout()
        plt.show()
        
        print("\n✅ Results visualized")
    else:
        print("⚠️ Water depth variable 'zs' not found")

⚠️ No simulation output found
Run the previous cell to execute the simulation
